In [1]:
# ЯЧЕЙКА 1. Импорты и ключи

import pandas as pd
import requests
import json
import time
from datetime import datetime
import os
from typing import List, Dict
from dotenv import load_dotenv

# ЛИБО читаем из .env:
# DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
# YANDEX_API_KEY = os.getenv("YANDEX_API_KEY")
# YANDEX_FOLDER_ID = os.getenv("YANDEX_FOLDER_ID")

# ЛИБО временно прописываем ключи руками (для отладки):
DEEPSEEK_API_KEY = "sk-ddf286ba8a304e6088c9843d83937624"
YANDEX_API_KEY = "AQVN223PO7EC_sImUpzXUamE8KmEZkriZvI30HF2"
YANDEX_FOLDER_ID = "b1gmcr1s1463agd1dkic" 

print("DeepSeek key:", "OK" if DEEPSEEK_API_KEY else "NOT FOUND")
print("Yandex key :", "OK" if YANDEX_API_KEY else "NOT FOUND")
print("Folder ID  :", "OK" if YANDEX_FOLDER_ID else "NOT FOUND")


DeepSeek key: OK
Yandex key : OK
Folder ID  : OK


In [2]:
# ЯЧЕЙКА 2. Загрузка JSON‑промптов и CSV‑отзывов

def load_prompts_from_json(filepath: str = "prompts.json") -> Dict[str, str]:
    """
    Загружает промпты из JSON файла.
    Ожидаемый формат:
    {
      "zero-shot": " ... {review_text} ... ",
      "few-shot":  " ... {review_text} ... ",
      "cot":       " ... {review_text} ... "
    }
    """
    with open(filepath, "r", encoding="utf-8") as f:
        prompts = json.load(f)
    print(f"✓ Промпты загружены из {filepath}. Варианты: {list(prompts.keys())}")
    return prompts


def load_reviews_from_csv(filepath: str = "reviews.csv") -> pd.DataFrame:
    df = pd.read_csv(filepath,sep=";", encoding="cp1251")  # или encoding="windows-1251"
    print(f"✓ Загружено {len(df)} отзывов из {filepath}")
    print("Колонки:", list(df.columns))
    return df


In [3]:
#ЯЧЕЙКА 3. Вспомогательные функции
# ЯЧЕЙКА 3. Вспомогательные функции

def substitute_review_in_prompt(prompt_template: str, review_text: str) -> str:
    """Подставляет текст отзыва в {review_text} внутри промпта."""
    return prompt_template.replace("{review_text}", review_text)


def extract_sentiment(result_text: str) -> str:
    text = (result_text or "").strip().lower()
    # print("RAW RESULT:", repr(text))  # можно оставить для проверки

    # ищем по всему тексту, без разницы где
    if "негатив" in text or "negative" in text:
        return "негатив"
    if "позитив" in text or "положительн" in text or "positive" in text:
        return "позитив"
    if "нейтр" in text or "neutral" in text:
        return "нейтрал"
    return "unknown"





In [4]:
# ЯЧЕЙКА 4. Запросы к моделям

def send_to_deepseek(prompt: str, use_reasoner: bool = False) -> Dict:
    """
    Отправка промпта в DeepSeek.
    use_reasoner=True → deepseek-reasoner (CoT),
    False → deepseek-chat.
    """
    if not DEEPSEEK_API_KEY:
        return {"status": "error", "error": "DEEPSEEK_API_KEY not set"}

    model = "deepseek-reasoner" if use_reasoner else "deepseek-chat"

    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "max_tokens": 200
    }

    try:
        r = requests.post(
            "https://api.deepseek.com/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=30
        )
        if r.status_code != 200:
            return {"status": "error", "error": f"HTTP {r.status_code}: {r.text[:200]}"}
        data = r.json()
        text = data["choices"][0]["message"]["content"]
        return {
            "status": "success",
            "model": "DeepSeek",
            "result": text,
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        return {"status": "error", "error": str(e)}


def send_to_yandex_gpt(prompt: str) -> Dict:
    """Отправка промпта в Yandex GPT."""
    if not YANDEX_API_KEY or not YANDEX_FOLDER_ID:
        return {"status": "error", "error": "YANDEX_API_KEY or YANDEX_FOLDER_ID not set"}

    headers = {
        "Authorization": f"Bearer {YANDEX_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model_uri": f"gpt://{YANDEX_FOLDER_ID}/yandexgpt-lite",
        "completion_options": {
            "stream": False,
            "temperature": 0.3,
            "max_tokens": 200
        },
        "messages": [{"role": "user", "text": prompt}]
    }

    try:
        r = requests.post(
            "https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
            headers=headers,
            json=payload,
            timeout=30
        )
        if r.status_code != 200:
            return {"status": "error", "error": f"HTTP {r.status_code}: {r.text[:200]}"}
        data = r.json()
        text = data["result"]["alternatives"][0]["message"]["text"]
        return {
            "status": "success",
            "model": "Yandex GPT",
            "result": text,
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        return {"status": "error", "error": str(e)}


In [5]:
# ЯЧЕЙКА 5. Основная функция обработки всех отзывов

def process_all_reviews(
    reviews_df: pd.DataFrame,
    prompts: Dict[str, str],
    models: List[str] = ["deepseek"],
    variants: List[str] = ["zero-shot", "few-shot", "cot"],
    limit: int = None,
    delay: float = 1.5
) -> pd.DataFrame:
    """
    Обрабатывает отзывы и возвращает DataFrame с результатами.
    Ожидаются колонки: 'id', 'отзыв', 'тональность'.
    Для КАЖДОГО запроса добавляется строка, даже если была ошибка.
    """
    if limit:
        reviews_df = reviews_df.head(limit)

    results = []
    total = len(reviews_df) * len(variants) * len(models)
    done = 0
    errors = 0

    print(f"Всего запросов: {total}")

    for idx, row in reviews_df.iterrows():
        review_id = row.get("id", idx)
        review_text = str(row.get("отзыв", ""))
        true_sentiment = row.get("тональность", None)

        print(f"\nОтзыв {idx + 1}/{len(reviews_df)} (ID={review_id}): {review_text[:70]}...")

        for variant in variants:
            if variant not in prompts:
                print(f"  ✗ Пропуск варианта '{variant}' — нет в prompts")
                continue

            prompt_template = prompts[variant]
            full_prompt = substitute_review_in_prompt(prompt_template, review_text)

            for model in models:
                # ВЫЗОВ МОДЕЛЕЙ
                if model.lower() == "deepseek":
                    use_reasoner = (variant == "cot")
                    resp = send_to_deepseek(full_prompt, use_reasoner)
                elif model.lower() == "yandex":
                    resp = send_to_yandex_gpt(full_prompt)
                else:
                    resp = {"status": "error", "error": f"Unknown model: {model}"}

                # ВСЕГДА ДОБАВЛЯЕМ СТРОКУ В results
                if resp.get("status") == "success":
                    raw_text = resp.get("result", "")
                    pred = extract_sentiment(raw_text)
                    status = "ok"
                    error_msg = ""
                    print(f"  ✓ {resp['model']}/{variant}: {pred}")
                else:
                    raw_text = resp.get("error", "")
                    pred = "unknown"
                    status = "error"
                    error_msg = raw_text[:200]
                    errors += 1
                    print(f"  ✗ {model}/{variant}: {error_msg}")

                results.append({
                    "review_id": review_id,
                    "review_text": review_text[:200],
                    "true_sentiment": true_sentiment,
                    "model": resp.get("model", model),
                    "variant": variant,
                    "prediction": pred,
                    "full_result": raw_text,
                    "status": status,
                    "error": error_msg,
                    "timestamp": resp.get("timestamp", datetime.now().isoformat()),
                })

                done += 1
                time.sleep(delay)

    df_res = pd.DataFrame(results)
    print("\nГотово.")
    print(f"Успешно: {done - errors}/{total}, ошибок: {errors}")
    return df_res



In [6]:
# ЯЧЕЙКА 6. Запуск теста / эксперимента

reviews_df = load_reviews_from_csv("reviews.csv")
prompts = load_prompts_from_json("prompts.json")

# 2. Полный запуск: 50 отзывов × 3 промпта × 2 модели
results = process_all_reviews(
    reviews_df=reviews_df,
    prompts=prompts,
    models=["deepseek", "yandex"],                # ← ОБЕ модели
    variants=["zero-shot", "few-shot", "cot"],    # ← ТРИ варианта промптов
    limit=50,                                     # ← ПЕРВЫЕ 50 отзывов
    delay=1.5                                     # пауза между запросами (можно 2.0)
)

results.head()


✓ Загружено 50 отзывов из reviews.csv
Колонки: ['id', 'отзыв', 'тональность']
✓ Промпты загружены из prompts.json. Варианты: ['zero-shot', 'few-shot', 'cot']
Всего запросов: 300

Отзыв 1/50 (ID=1): Добрый день! Не рекомендую выбирать Нетологию, т к по сути вы покупает...
  ✓ DeepSeek/zero-shot: негатив
  ✓ Yandex GPT/zero-shot: негатив
  ✓ DeepSeek/few-shot: негатив
  ✓ Yandex GPT/few-shot: негатив
  ✓ DeepSeek/cot: unknown
  ✓ Yandex GPT/cot: негатив

Отзыв 2/50 (ID=2): Прошла обучение и получила сертификат в 2023 году – полное разочарован...
  ✓ DeepSeek/zero-shot: негатив
  ✓ Yandex GPT/zero-shot: негатив
  ✓ DeepSeek/few-shot: негатив
  ✓ Yandex GPT/few-shot: негатив
  ✓ DeepSeek/cot: unknown
  ✓ Yandex GPT/cot: негатив

Отзыв 3/50 (ID=3): Прохожу курс «Системный аналитик» от Нетологии и хочу поделиться своим...
  ✓ DeepSeek/zero-shot: позитив
  ✓ Yandex GPT/zero-shot: позитив
  ✓ DeepSeek/few-shot: позитив
  ✓ Yandex GPT/few-shot: позитив
  ✓ DeepSeek/cot: unknown
  ✓ Yandex GPT/c

,review_id,review_text,true_sentiment,model,variant,prediction,full_result,status,error,timestamp
0,1,"Добрый день! Не рекомендую выбирать Нетологию,...",негатив,DeepSeek,zero-shot,негатив,негатив,ok,,2025-12-12T19:54:41.594769
1,1,"Добрый день! Не рекомендую выбирать Нетологию,...",негатив,Yandex GPT,zero-shot,негатив,негатив\n,ok,,2025-12-12T19:54:43.465741
2,1,"Добрый день! Не рекомендую выбирать Нетологию,...",негатив,DeepSeek,few-shot,негатив,негатив,ok,,2025-12-12T19:54:46.148814
3,1,"Добрый день! Не рекомендую выбирать Нетологию,...",негатив,Yandex GPT,few-shot,негатив,негатив\n\nВопрос 1\n\nВопрос: Какие аспекты к...,ok,,2025-12-12T19:54:50.095165
4,1,"Добрый день! Не рекомендую выбирать Нетологию,...",негатив,DeepSeek,cot,unknown,,ok,,2025-12-12T19:54:58.288306


In [7]:
# ЯЧЕЙКА 8 — сохранение в CSV
results.to_csv("results.csv", index=False, encoding="utf-8")

In [8]:
from sklearn.metrics import accuracy_score, classification_report

ds = df[(df["model"] == "DeepSeek") & (df["status"] == "ok")]

for v in ["zero-shot", "few-shot", "cot"]:
    cur = ds[ds["variant"] == v]
    print("\n=== Variant:", v, "===")
    print("Accuracy:", accuracy_score(cur["true_sentiment"], cur["prediction"]))
    print(classification_report(cur["true_sentiment"], cur["prediction"]))


NameError: name 'df' is not defined

In [9]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# 1. Загружаем файл с результатами
df = pd.read_csv("results.csv", encoding="utf-8")

# 2. Берём только DeepSeek и успешные ответы
ds = df[(df["model"] == "DeepSeek") & (df["status"] == "ok")]

# 3. Считаем метрики по каждому типу промпта
for v in ["zero-shot", "few-shot", "cot"]:
    cur = ds[ds["variant"] == v]
    print("\n=== Variant:", v, "===")
    print("Accuracy:", accuracy_score(cur["true_sentiment"], cur["prediction"]))
    print(classification_report(cur["true_sentiment"], cur["prediction"]))



=== Variant: zero-shot ===
Accuracy: 0.88
              precision    recall  f1-score   support

     негатив       0.81      1.00      0.89        17
     нейтрал       0.91      0.67      0.77        15
     позитив       0.94      0.94      0.94        18

    accuracy                           0.88        50
   macro avg       0.89      0.87      0.87        50
weighted avg       0.89      0.88      0.87        50


=== Variant: few-shot ===
Accuracy: 0.94
              precision    recall  f1-score   support

     негатив       1.00      0.94      0.97        17
     нейтрал       0.83      1.00      0.91        15
     позитив       1.00      0.89      0.94        18

    accuracy                           0.94        50
   macro avg       0.94      0.94      0.94        50
weighted avg       0.95      0.94      0.94        50


=== Variant: cot ===
Accuracy: 0.0
              precision    recall  f1-score   support

     unknown       0.00      0.00      0.00       0.0
     нег

C:\Users\Alinochka\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Alinochka\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Alinochka\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Alinochka\anaconda3\Lib\sit

In [4]:
!pip install chardet


In [5]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# 1. Загружаем файл с результатами
df = pd.read_csv("results.csv", encoding="utf-8")

# 2. Для каждой модели и каждого типа промпта считаем метрики
for model_name in ["DeepSeek", "Yandex GPT"]:
    print("\n\n############################")
    print("МОДЕЛЬ:", model_name)
    print("############################")

    mdf = df[(df["model"] == model_name) & (df["status"] == "ok")]

    for v in ["zero-shot", "few-shot", "cot"]:
        cur = mdf[mdf["variant"] == v]
        if cur.empty:
            print(f"\n=== Variant: {v} ===")
            print("Нет данных (0 успешных ответов).")
            continue

        print(f"\n=== Variant: {v} ===")
        print("Accuracy:", accuracy_score(cur["true_sentiment"], cur["prediction"]))
        print(classification_report(cur["true_sentiment"], cur["prediction"]))




############################
МОДЕЛЬ: DeepSeek
############################

=== Variant: zero-shot ===
Accuracy: 0.78
              precision    recall  f1-score   support

     негатив       0.85      1.00      0.92        17
     нейтрал       1.00      0.35      0.52        17
     позитив       0.67      1.00      0.80        16

    accuracy                           0.78        50
   macro avg       0.84      0.78      0.75        50
weighted avg       0.84      0.78      0.75        50


=== Variant: few-shot ===
Accuracy: 0.9
              precision    recall  f1-score   support

     негатив       1.00      0.94      0.97        17
     нейтрал       0.93      0.76      0.84        17
     позитив       0.80      1.00      0.89        16

    accuracy                           0.90        50
   macro avg       0.91      0.90      0.90        50
weighted avg       0.91      0.90      0.90        50


=== Variant: cot ===
Accuracy: 0.0
              precision    recall  f1-sco

C:\Users\Alinochka\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Alinochka\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Alinochka\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Alinochka\anaconda3\Lib\sit

In [11]:
import chardet


def convert_to_utf8(input_path, output_path):
    # 1. Определяем кодировку
    with open(input_path, "rb") as f:
        data = f.read()  # можно f.read(1000000) для больших файлов
    detected = chardet.detect(data)
    src_enc = detected["encoding"]
    print(f"Определена кодировка: {src_enc}")

    # 2. Читаем в исходной кодировке и сохраняем в UTF-8
    with open(input_path, "r", encoding=src_enc, errors="ignore") as infile:
        text = infile.read()
    with open(output_path, "w", encoding="utf-8", newline="") as outfile:
        outfile.write(text)
    print(f"Файл сохранён как UTF-8: {output_path}")



In [13]:
convert_to_utf8("results.csv", "res_utf8.csv")
# или
# convert_to_utf8("input.txt", "input_utf8.txt")


Определена кодировка: utf-8
Файл сохранён как UTF-8: res_utf8.csv
